In [1]:
# %% allow reloading of modules
%load_ext autoreload
%autoreload 2

#### Note: ensure the you are in the `notebooks` directory when using the `jupyter notebook` command 

In [2]:
import os
path_parent = os.path.dirname(os.getcwd())
os.getcwd()

'/home/joemcormier/pycrash/projects/validation - single vehicle motion/notebooks'

In [3]:
import sys
#sys.path.append("/home/jmc/Documents/pycrash")
sys.path.append("/home/joemcormier/pycrash/")

In [4]:
import pycrash
from pycrash.project import Project, project_info, load_project
from pycrash.vehicle import Vehicle
from pycrash.kinematics import SingleMotion

Current values for defined constants:
maximum available friction (mu_max) = 0.9
time step for vehicle motion (dt) = 0.1 s
No Environment File Provided
No Environment File Provided


In [5]:
os.getcwd()

'/home/joemcormier/pycrash/projects/validation - single vehicle motion'

In [6]:
import pandas as pd
import numpy as np
import math
from scipy import integrate
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from IPython import get_ipython
from IPython.display import display
get_ipython().run_line_magic('matplotlib', 'qt')

### Load Project Datafile that was created when project was instantiated
- the project object contains information needed to run other parts of Pycrash

#### Get project information - used to import project object

In [7]:
project_info('validation - single vehicle motion')

This saved project contains:
Object of type "project" with name "validation - single vehicle motion"
list objects in this order for loading project: ['validation - single vehicle motion']
Example: project_name, veh1, veh2 = load_project('project_name')


#### Load project and print contents

In [8]:
proj = load_project('validation - single vehicle motion')
proj.show()

----------------------------------  ------------------------  -----------  ---------------  --------------------
Project                             Description               Impact Type  Simulation Type  Note
validation - single vehicle motion  planar motion simulation               SV               pc crash comparisons
----------------------------------  ------------------------  -----------  ---------------  --------------------


In [9]:
proj.__dict__

{'name': 'validation - single vehicle motion',
 'project_path': '/home/jmc/Documents/pycrash/projects/',
 'pdesc': 'planar motion simulation',
 'sim_type': 'SV',
 'impact_type': None,
 'note': 'pc crash comparisons',
 'type': 'project'}

### Generate dataframe with driver inputs
- use validation steer / brake / throttle inputs
- inputs are same dt as pycrash will have to check in future

In [10]:
pc_crash_column_names = ['t', 'ax', 'ay', 'az', 'phi_deg', 'lf_fy', 'rf_fy',
                         'lr_fy', 'rr_fy', 'delta_deg', 'rf_delta_deg', 'steer', 
                         'steer_rate', 'X', 'Y', 'Z', 'roll', 'pitch', 'theta_deg',
                         'Vx', 'Vy', 'Vz']

In [11]:
df = pd.read_excel(os.path.join(path_parent, 'data', 'external', '15-mph-steer-data.xlsx'),
                            na_filter = False, header = None, names = pc_crash_column_names, skiprows = 1)

In [12]:
# convert velocities to fps
df.Vx = [x * 1.46667 for x in df.Vx]
df.Vy = [x * 1.46667 for x in df.Vy]
df.Vz = [x * 1.46667 for x in df.Vz]

# convert acceleration to fps/s
df.ax = [x * 32.2 for x in df.ax]
df.ay = [x * 32.2 for x in df.ay]
df.az = [x * 32.2 for x in df.az]

# convert tire forces to lb
df.lf_fy = [x * 1000 for x in df.lf_fy]
df.rf_fy = [x * 1000 for x in df.rf_fy]
df.lr_fy = [x * 1000 for x in df.lr_fy]
df.rr_fy = [x * 1000 for x in df.rr_fy]

# steer angle in radians
df['delta_rad'] = [x / 180 * math.pi for x in df.delta_deg]

In [13]:
# integrate velocities to get displacements
df['Dx'] = 0 + integrate.cumtrapz(list(df.Vx), list(df.t), initial=0)
df['Dy'] = 0 + integrate.cumtrapz(list(df.Vy), list(df.t), initial=0)
df.head()

,t,ax,ay,az,phi_deg,lf_fy,rf_fy,lr_fy,rr_fy,delta_deg,rf_delta_deg,steer,steer_rate,X,Y,Z,roll,pitch,theta_deg,Vx,Vy,Vz,delta_rad,Dx,Dy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.79,0.0,0.0,0.0,22.00005,0.0,0.0,0.0,0.000000,0.0
1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2.2,0.0,1.79,0.0,0.0,0.0,22.00005,0.0,0.0,0.0,2.200005,0.0
2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4.4,0.0,1.79,0.0,0.0,0.0,22.00005,0.0,0.0,0.0,4.400010,0.0
3,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,6.6,0.0,1.79,0.0,0.0,0.0,22.00005,0.0,0.0,0.0,6.600015,0.0
4,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,8.8,0.0,1.79,0.0,0.0,0.0,22.00005,0.0,0.0,0.0,8.800020,0.0


In [14]:
df.plot(x = 't', y = 'steer')

<AxesSubplot:xlabel='t'>

In [15]:
df.t.max()

10.9

In [16]:
df.Vx[0]

22.000049999999998

### Load Validation Data

In [17]:
# validation vehicle data

vehicle_input_dict = {"year":2004,
"make":"Chevrolet",
"model":"Malibu",
"weight":3298,
"vin":"1G1ZU54854F135916",
"brake":0,
"steer_ratio":15.9,
"init_x_pos":0,
"init_y_pos":0,
"head_angle":0,
"width":70 / 12,
"length":187 / 12,
"hcg":21.5 / 12,
"lcgf":38.1 / 12,
"lcgr":67.9 / 12,
"wb":106 / 12,
"track":60 / 12,
"f_hang":38 / 12,
"r_hang":43 / 12,
"tire_d":26.2 / 12,
"tire_w":8.5 / 12,
"izz":2040,
"fwd":1,
"rwd":0,
"awd":0,
"A":100,
"B":41,
"k":1000,
"L":0,
"c":0,
"vx_initial":df.Vx[0],
"vy_initial":0,
"omega_z":0}

In [18]:
end_time = df.t.max()  
t = list(df.t)  
throttle = [0] * len(t)                      
brake = [0] * len(t)                         
steer = list(df.steer)                                   
driver_input_dict = {'t':t, 'throttle':throttle, 'brake':brake, 'steer':steer}
driver_input_df = pd.DataFrame.from_dict(driver_input_dict)
print('Vehicle Driver Inputs:')
driver_input_df.head()

Vehicle Driver Inputs:


,t,throttle,brake,steer
0,0.0,0,0,0.0
1,0.1,0,0,0.0
2,0.2,0,0,0.0
3,0.3,0,0,0.0
4,0.4,0,0,0.0


In [19]:
driver_input_df.t.iloc[-1]

10.9

### Assign validation data to veh2

In [20]:
veh2 = Vehicle('Veh2', vehicle_input_dict)
veh2.driver_input = driver_input_df
# apply validation data as a model result
veh2.model = df

Vehicle inputs for Veh2 applied succesfully


### Create Pycrash vehicle:
- "Vehicle" stores information about a single vehicle - all possible inputs do not need to be entered
- creating a Vehicle requries a "name" which is used to identify the vehicle in outputs / plots etc. 

In [35]:
veh1 = Vehicle('Veh1', vehicle_input_dict)
veh1.driver_input = driver_input_df
veh1.vx_initial = 15
veh1.hcg = 0   # vary cg height

Vehicle inputs for Veh1 applied succesfully


### Create Single Motion Instance
- simulation is run when instance is created

In [36]:
run = SingleMotion('15_mph_steer', veh1)

Maximum allowable friction: 0.76
Time step for vehicle motion (s) : 0.1
Maximum tire slip angle (deg): 10.00
Driver input for Veh1 of shape = (110, 4)
Vehicle motion will be simulated for 10.9 seconds


In [37]:
run.veh.hcg

0

### View Driver Inputs

In [38]:
run.plot_inputs()

### Plot Motion

In [39]:
run.plot_model()

In [40]:
#run.veh.model.plot(x = 't', y = 'ay')

<AxesSubplot:xlabel='t'>

### Plot Vehicle Motion

In [27]:
len(run.veh.model.t)

110

In [28]:
run.veh.model.t.iloc[-1]

10.9

In [42]:
run.global_motion(109)

### Compare to PC Crash Model

In [43]:
from pycrash.visualization.kinematics_compare import compare_kinematics

In [44]:
# calculate vehicle slip angle for pycrash model - need to add
phi_rad = []
phi_deg = []
for i in range(len(run.veh.model.t)):
    phi_rad.append(math.atan2(run.veh.model.vy[i], run.veh.model.vx[i]) * -1) 
    phi_deg.append(math.atan2(run.veh.model.vy[i], run.veh.model.vx[i]) * (180 / math.pi)  * -1 )
    
run.veh.model['phi_rad'] = phi_rad
run.veh.model['phi_deg'] = phi_deg

In [45]:
compare_kinematics(run.veh.model, df, 'pycrash', 'validate')

In [46]:
from pycrash.visualization.tire_details import tire_details, vertical_forces

In [47]:
tire_details(run.veh)

In [48]:
vertical_forces(run.veh)

### Get position data for validation vehicle

In [49]:
from pycrash.visualization.cg_motion_compare import cg_motion

In [50]:
cg_motion(run.veh.model, df, 'pycrash', 'validate')